In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kubig-2024-summer/.parquet


In [2]:
data = pd.read_parquet("/kaggle/input/kubig-2024-summer/.parquet")

In [3]:
data

,speaker_1,speaker_2,speaker_2_eojeol_sum
0,자 음식을 멀 좋아하느냐 하면은 나는 사실은 다 좋아합니다 다 좋아하는데 특히 육류...,아 자기 아 오리를 싫어했었구나,0
1,머 못 먹 그리고 모든걸 다 좋아하는 데 즐기진 않는다,즐기진 않는다 자기는 나하고 반대네 나는 야채를 좋아하거든,0
3,머라노 그 곁들어 먹는,곁들어먹는거 근데 야채를 많이 먹어야 건강해요,0
5,그래서 음식에 가린다는 것은 나는 한편으로 불행하다 생각합니다 아니 어쩔수 없이 먹...,그러면 사실은 우리는 뭐 없어서 못 묵지,1
7,밥을 챙겨먹는 생각도 의미 없다고 생각해 영양분이 되고 배가 부르면은 한 끼 식사로...,근데 나는 그기서 쪼끔 반대하는기 머냐면 이 인제 밥이 탄수화물이잖아 탄수화물인데 ...,2
...,...,...,...
113814,요즘에 트로트가 진짜 많이 이렇게 대세잖아요 방송 막 틀어보면 뭐 방송 상사 뭐 삼...,아 내가 임영웅을 알았는 거는 미스터 트롯 그 프로를 보면서 알았는데 야가 이 사람...,93
113815,별로 저는 그거 노래 그렇게 막 잘하 아 잘하는 건 알겠는데 근데 막 계속 보고 싶...,내가 그 당시에는 임영웅 영웅이한테 빠져 갖고 다른 사람이 안 보여 가주고 몰랐는데...,64
113816,근데 나는 엄마가 그렇게 누군가를 좋아하고 관심있게 보고 뭐 이러는 게 진짜 조금 ...,배우 중에는 내가 영화고 드라마고 이런 거를 잘 안 보기 때문에 나는 좋아하는 사람...,7
113817,나는 요즘에는 유아인 예전에 그 사도나 배테랑 그때 이천십오 년도에 개봉했을 때 그...,개그맨은 옛날에는 우리가 그거 저기 개그 프로 근데 뭐 제목은 내가 모르겠는데 뭐 ...,26


In [4]:
# basic EDA

data_2 = data.copy()

data_2["speaker_1_length"] = data_2["speaker_1"].apply(lambda x : len(x.split(" ")))
data_2["speaker_2_length"] = data_2["speaker_2"].apply(lambda x : len(x.split(" ")))

speaker_1_length = list(data_2["speaker_1_length"])
speaker_2_length = list(data_2["speaker_2_length"])

quartiles_1 = np.percentile(speaker_1_length, [25, 50, 75])
quartiles_2 = np.percentile(speaker_2_length, [25, 50, 75])
quartiles_3 = np.percentile(data_2["speaker_2_eojeol_sum"], [25, 50, 75])

# 사분위수
print(quartiles_1)
print(quartiles_2)
print(quartiles_3)

print(len(data_2[data_2["speaker_2_eojeol_sum"] == 0]))

[11. 32. 99.]
[ 11.  34. 103.]
[0. 1. 2.]
45960


In [5]:
# data_2 = data_2[data_2["speaker_1_length"] > 11]
# data_2 = data_2[data_2["speaker_2_length"] > 11]

# data_2 = data_2[data_2["speaker_1_length"] < 129]
# data_2 = data_2[data_2["speaker_2_length"] < 129]

# data_2 = data_2[data_2["speaker_2_eojeol_sum"] > 3]

In [6]:
data_2 = data_2[data_2["speaker_2_eojeol_sum"] > 5]

In [7]:
data_2 = data_2[["speaker_1", "speaker_2"]]
data_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9975 entries, 10 to 113818
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   speaker_1  9975 non-null   object
 1   speaker_2  9975 non-null   object
dtypes: object(2)
memory usage: 233.8+ KB


In [8]:
import pandas as pd
import torch

from transformers import T5TokenizerFast, T5ForConditionalGeneration
from transformers import AdamW

from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset

# 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        questions = self.data.iloc[idx]['speaker_1']
        answers = self.data.iloc[idx]['speaker_2']

        # 토큰 수를 max_length에 맞춰서 자르거나 패딩
        inputs = self.tokenizer(questions, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        labels = self.tokenizer(answers, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')

        return {
            'input_ids': inputs.input_ids[0],
            'labels': labels.input_ids[0]
        }


# 토크나이저와 모델 로드
# tokenizer = T5TokenizerFast.from_pretrained('paust/pko-chat-t5-base')
# model = T5ForConditionalGeneration.from_pretrained('paust/pko-chat-t5-base')

tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')
model = T5ForConditionalGeneration.from_pretrained('paust/pko-t5-base')
model.to(device)


# tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')
# model = T5ForConditionalGeneration.from_pretrained('paust/pko-t5-base')

# 학습 설정
optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 5
log_interval = 100
checkpoint_interval = 0.5  # 0.5 에폭마다 체크포인트 저장

# DataLoader를 사용하여 데이터 로드
batch_size = 4
dataset = CustomDataset(data_2, tokenizer)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 학습 루프에서 데이터 로더 사용
total_steps = 0
for epoch in range(num_epochs):
    for batch in tqdm(data_loader, total=len(data_loader)):
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        # 모델 학습
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_steps += 1

        # 0.5 에폭마다 체크포인트 저장
        if total_steps % int(len(dataset) / (batch_size * 2)) == 0:
            checkpoint_dir = f"checkpoint_{total_steps}"
            model.save_pretrained(checkpoint_dir)

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.90M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 2494/2494 [08:06<00:00,  5.12it/s]


In [9]:
# 모델 상태 사전을 저장할 경로 지정
save_path = '/kaggle/working/t5_finetuning_3.pth'

# 모델 상태 사전 저장
torch.save(model.state_dict(), save_path)

# # 저장된 상태 사전 로드
# model.load_state_dict(torch.load('t5_finetuning.pth'))

In [10]:
# pth 파일 다운로드를 위한 코드

import os
os.chdir(r'/kaggle/working')

# %cd /kaggle/working
# Now save your required files in this Directory .
# Then run the following cell
from IPython.display import FileLink

FileLink(r't5_finetuning_3.pth')

/kaggle/working/t5_finetuning_3.pth

In [11]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(50358, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50358, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [12]:
input_text = "오늘 저녁은 뭘 먹을까?"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

In [13]:
# 문장 생성
generated_ids = model.generate(
    input_ids=input_ids,
    max_length=128,  # 생성할 문장의 최대 길이 설정
    num_beams=5,  # 빔 서치에서 사용할 빔 개수 (선택 사항)
    early_stopping=True  # 조기 종료 (선택 사항)
)

# 생성된 토큰을 텍스트로 변환
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated text:", generated_text)

Generated text: 오늘 저녁은 또 뭘 먹지 고민이네  오늘 저녁은 또 뭘 먹을까 고민이네  오늘 저녁은 또 뭘 먹을까 고민이네  오늘 저녁은 또 뭘 먹을까 고민이네  오늘 저녁은 또 뭘 먹을까 고민이네  오늘 저녁은 또 뭘 먹을까 고민이네  오늘 저녁은 또 뭘 먹을까 고민이네  오늘 저녁은 또 뭘 먹지 고민이네  오늘 저녁은 또 


In [14]:
# 문장 생성 2
generated_ids = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=10,
    top_p=0.95,
    num_return_sequences=3
)

# 생성된 토큰을 텍스트로 변환
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated text:", generated_text)

Generated text: 오늘 저녁은 대체 뭘 먹을까 고민이네 음 오늘 저녁은 왠지 치킨을 먹어야 되지 않을까 싶으네 음 그럼 난 오늘은 저녁은 또 


In [15]:
input_text = "남북한 관계에 대해서 어떻게 생각해?"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

In [16]:
# 문장 생성
generated_ids = model.generate(
    input_ids=input_ids,
    max_length=128,  # 생성할 문장의 최대 길이 설정
    num_beams=5,  # 빔 서치에서 사용할 빔 개수 (선택 사항)
    early_stopping=True  # 조기 종료 (선택 사항)
)

# 생성된 토큰을 텍스트로 변환
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated text:", generated_text)

Generated text: 남북한 관계에 대해서 깊게 생각해본 적은 없지만 그래도 한 번씩 생각은 한 번씩은 생각은 한 번씩은 생각은 한 번씩은 생각은 한 번씩은 생각은 한 번씩은 생각은 한 번씩은 생각은 한 번씩은 생각은 한 번씩은 생각은 한 번씩은 생각은 한 번씩은 해 보는 편이고 그렇게 해 본 


In [17]:
# 문장 생성 2
generated_ids = model.generate(
    input_ids,
    do_sample=True,
    max_length=128,
    top_k=10,
    top_p=0.95,
    num_return_sequences=3
)

# 생성된 토큰을 텍스트로 변환
print("Output:\n" + 100 * '-')
for i, generated_id in enumerate(generated_ids):
  print("{}: {}".format(i, tokenizer.decode(generated_id.tolist(), skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------


2024-08-20 10:26:05.910130: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 10:26:05.910259: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 10:26:06.021941: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


0: 저도 그 같은 생각을 가지고 있거든요 저는 먼저 그 햇볕 정책에 대해서 정말 정말 깊이 공감을 하고 정말 그 정말 그 저는 그 저 정말 그 생각을 깊이 공감을 하고 정말 그 이해는 또 이해는 하지만 정말 이해는 하지만 그 이해는 하지만 정말 그 생각 자체가 좀 다른 것 같다고 생각해요 저는 물론 그 정책들이 
1: 나는 솔직히 말해서 분단은 솔직히 너무 안타까운 일이라고 생각을 하고 그리고 그것이 현실화 될 수밖에 없기 때문에 그렇게 갈라진 거라고 생각을 하거든 그리고 그 이후에 뭐 중국인가 소련인가 뭐 그런 쪽에 그 부분은 솔직히 말해서 문제시 되지 않는다고 생각해 개인적으로는 개인적으로는 중국이 정말 나쁘다고 생각해 개인적으로는 근데 또 개인적으로는 소련이 정말
2: 북한에는 김일성이가 정말 철혈 외교 전술을 잘 알고 있어 그래서 앞으로 통일이 되면 어떻게 될 것인지 정말 굉장히 잘 알고 있네 그다음  뭐 그다음에 그다음  뭐 김일성과 그 가족들 그다음 김일성 형제 그다음 가족들 우리 아들들이 정말 철혈 관계에 있어서 철혈 관계 잘 알고 있네 그다음  뭐 천구백구십 키로 


In [18]:
input_text = "저녁 메뉴 추천해줘"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

In [19]:
# 문장 생성
generated_ids = model.generate(
    input_ids=input_ids,
    max_length=128,  # 생성할 문장의 최대 길이 설정
    num_beams=5,  # 빔 서치에서 사용할 빔 개수 (선택 사항)
    early_stopping=True  # 조기 종료 (선택 사항)
)

# 생성된 토큰을 텍스트로 변환
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated text:", generated_text)

Generated text: 오늘  오늘  목요일 저녁에 먹어 목요일 저녁에 먹을 저녁 메뉴 추천해줘 목요일 저녁에 먹을 저녁 메뉴 추천해줘 아니면 오늘 저녁에 먹을 저녁 메뉴랑 금요일 저녁에 먹을 저녁 메뉴 추천해줘 아니면 토요일 저녁에 먹을 저녁 메뉴 추천해줘 토요일 저녁에 먹을 점심 메뉴 추천해줘 아니면 일요일 점심에 먹을 점심 메뉴 추천해 일요일 점심에 


In [20]:
# 문장 생성 2
generated_ids = model.generate(
    input_ids,
    do_sample=True,
    max_length=128,
    top_k=10,
    top_p=0.95,
    num_return_sequences=3
)

# 생성된 토큰을 텍스트로 변환
print("Output:\n" + 100 * '-')
for i, generated_id in enumerate(generated_ids):
  print("{}: {}".format(i, tokenizer.decode(generated_id.tolist(), skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: 오늘  나 월요일 오후 근무야 월요일 저녁 메뉴 결정 못하는데 월요일 저녁 메뉴로 괜찮은 시원한 아메리카나 시원한 아메리카나 있으면 점심때 시켜 먹고 저녁에 먹을 메뉴는 아직 없어 그럼 점심에는 뭐 먹을까 고민하다가 아무래도 오늘 저녁은 메뉴를 좀 많이 못 고르겠어 그래서 메뉴 중에는 못 고르겠고 혹시 여자친구 있을라나 아니면은 남자친구 있을라나 모르겠는데 월요일 
1: 나 오늘  근데 오늘 저녁 메뉴 선택한 적 없는데 여자친구가 니 저녁 메뉴 피자 피자파스타면 피자 선택한 이유 물어본다 남자친구가 여자친구 저녁 메뉴로 피자 파스타면 어때 남자친구 취향이 분명한데 혹시 아 여자친구 추천 안 한 거 아니야 혹시 피자 파스타면 피자 파스타면 여자친구 오늘 저녁 메뉴 피자 파스타면 그걸로 저녁 때우고 싶다는데
2: 저녁에 갑자기 찜닭 먹고 싶어갖고 엄마 퇴근해갖고 찜닭 먹고 싶다고 해갖고 내가 허겁지겁 허겁지겁 먹고 싶어갖고 엄마 나 못 먹게 해준다고 해서 못 먹게 하지 마라 아 나 지금 너무 배가 안고파서 엄마 퇴근할 때 찜닭 먹을 건데 그럼 저녁에 찜닭 먹으면 안 돼 혹시 몰라 엄마 혹시 몰라 집에 갈 때 돼 가지고 아빠한테 말해 봤을 


In [21]:
input_text = "현대 사회의 가장 큰 문제가 뭐라고 생각해?"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

In [22]:
# 문장 생성
generated_ids = model.generate(
    input_ids=input_ids,
    max_length=128,  # 생성할 문장의 최대 길이 설정
    num_beams=5,  # 빔 서치에서 사용할 빔 개수 (선택 사항)
    early_stopping=True  # 조기 종료 (선택 사항)
)

# 생성된 토큰을 텍스트로 변환
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated text:", generated_text)

Generated text: 나는 개인적으로 요즘 사회 문제라고 하면 개인적으로 제일 먼저 생각하는 건 빈부 격차 아닐까 개인적으로는 요즘 사회의 가장 큰 문제라고 생각해 사회의 가장 큰 문제라고 하면 개인적으로는 빈부 격차 아닐까 개인적으로는 요즘 사회의 가장 큰 문제라고 생각해 사회의 가장 큰 문제라고 하면 개인적으로는 부 분배의 문제라고 생각해 그럼 개인적으로는 


In [23]:
# 문장 생성 2
generated_ids = model.generate(
    input_ids,
    do_sample=True,
    max_length=128,
    top_k=10,
    top_p=0.95,
    num_return_sequences=3
)

# 생성된 토큰을 텍스트로 변환
print("Output:\n" + 100 * '-')
for i, generated_id in enumerate(generated_ids):
  print("{}: {}".format(i, tokenizer.decode(generated_id.tolist(), skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: 내가 개인적으로 생각하기에는 자본주의가 성립 안 된 진짜 이유라고 생각을 하는 것들이 있는데 그게 어떻게 보면 딱 봤을 때 성립 안 된 자본주의의 산물이라고 생각을 하거든 그 자본주의가 어떻게 보면 어느 사회 속에서 살아가는 데 가장 필수적이라고 생각을 하는데 그럼에도 불구하고 자본주의가 성립 안 된 데에는 분명히 문제점들이 분명 존재한다고 생각을 하는데 
1: 나는 개인적으로 요즘 사회 문제라고 하면 정말 딱 떠오르는 게 빈부 격차 이거 아닐까 라고 생각하거든 사회 초년생 같은 사람들이 사회 초년생들에 비해서 생활이 어떤지 모르겠는데 사회 초년생들이 정말 생활하기 불편한 점이 생활하기에 불편하게 느껴지는 거야 인간의 존엄성에 대한 문제랄까 인간의 존엄성이란 건 정말 무시할 수 없는 그런 거라고 생각하거든 사회 
2: 난 개인적으로 인간의 존엄성에 대해서 생각을 하고 있다고 생각해 그게 물론 물질적으로 인간이 존엄해야 하지만 그럼에도 인간이 존엄 그 그 무엇일 지 그걸 구별하지 못하는 현 사회의 문제라고 생각을 하거든 그래서 나는 그런 윤리 관념이나 그런 사회 면 사회의 큰 문제라고 생각을 하고 있다고 생각을 해 개인적으로는 인간의 존엄성에 
